# 03. Mock Photon Comparison

In this Notebook we compare the mock photon propagation with the existing methods like the normal flow. To do that, we generate a line detector and a couple of events that we propagate using each propagator.

In [2]:
from ananke.configurations.presets.detector import single_line_configuration
from ananke.services.detector import DetectorBuilderService
from olympus.event_generation.generators import get_generator
from olympus.event_generation.photon_propagation.mock_photons import MockPhotonPropagator
from olympus.event_generation.photon_propagation.norm_flow_photons import NormalFlowPhotonPropagator
from olympus.configuration.generators import DatasetConfiguration
from olympus.configuration.photon_propagation import MockPhotonPropagatorConfiguration, NormalFlowPhotonPropagatorConfiguration

/home/janik/projects/master/env/lib/python3.8/site-packages/jax/_src/api_util.py:222: SyntaxWarning: Jitted function has invalid argnames {'interactions'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "
2023-01-30 13:36:44.213445: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
/home/janik/projects/master/env/lib/python3.8/site-packages/haiku/_src/data_structures.py:37: FutureWarning: jax.tree_structure is deprecated, and will be removed in a future release. Use jax.tree_util.tree_structure instead.
  PyTreeDef = type(jax.tree_structure(None))


Let's define the detector next:

In [3]:
detector_service = DetectorBuilderService()
det = detector_service.get(configuration=single_line_configuration)

In [4]:
det.df[['module_location_x', 'module_location_y', 'module_location_z']].drop_duplicates()

,module_location_x,module_location_y,module_location_z
0,0.0,0.0,0.0
16,0.0,0.0,50.0
32,0.0,0.0,100.0
48,0.0,0.0,150.0
64,0.0,0.0,200.0
80,0.0,0.0,250.0
96,0.0,0.0,300.0
112,0.0,0.0,350.0
128,0.0,0.0,400.0
144,0.0,0.0,450.0


Next up we generate our events:

In [7]:
configuration = DatasetConfiguration.parse_obj({
    'generator': {
        'type': 'cascade',
        'spectrum': {
            'log_minimal_energy': 2.0,
            'log_maximal_energy': 5.5,
        }
    },
    'number_of_samples': 2
})

cascades_generator = get_generator(
    detector=det,
    configuration=configuration.generator
)

records = cascades_generator.generate_records(
    number_of_samples=configuration.number_of_samples
)

sources = cascades_generator.propagate(records)

records.df.head()

,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,energy,length,time,type,particle_id
0,42.151271,-3.693700,-221.257582,-0.186077,-0.861640,-0.472178,-3639980863655374355,838.859127,3000.0,0.0,cascade,11
1,25.837680,12.317769,597.338950,-0.161914,-0.749751,-0.641605,-3639979918762569235,423.830054,3000.0,0.0,cascade,11


In [8]:
sources.df.head()

,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,time,number_of_photons,type
0,42.146339,-3.696871,-221.463745,-0.186077,-0.86164,-0.472178,-3639980863655374355,0.336900,832,cherenkov
1,42.038254,-3.858296,-221.550156,-0.186077,-0.86164,-0.472178,-3639980863655374355,1.004028,46121,cherenkov
2,42.093220,-4.064077,-221.619873,-0.186077,-0.86164,-0.472178,-3639980863655374355,1.671156,394202,cherenkov
3,41.849957,-4.267915,-221.575607,-0.186077,-0.86164,-0.472178,-3639980863655374355,2.338284,1548311,cherenkov
4,41.891029,-4.522918,-221.550034,-0.186077,-0.86164,-0.472178,-3639980863655374355,3.005413,4015380,cherenkov


Now we need our photon propagators

In [9]:
mock_photon_propagator_configuration = MockPhotonPropagatorConfiguration(resolution=18000)

mock_photon_propagator = MockPhotonPropagator(
    detector=det,
    configuration=mock_photon_propagator_configuration
)

normal_photon_propagator_configuration = NormalFlowPhotonPropagatorConfiguration(
    shape_model_path="../../hyperion/data/photon_arrival_time_nflow_params.pickle",
    counts_model_path="../../hyperion/data/photon_arrival_time_counts_params.pickle"
)

normal_photon_propagator = NormalFlowPhotonPropagator(
    detector=det,
    configuration=normal_photon_propagator_configuration
)

/home/janik/projects/master/env/lib/python3.8/site-packages/haiku/_src/data_structures.py:206: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, structure = jax.tree_flatten(mapping)


Now lets Propagate:

In [10]:
mock_hits = mock_photon_propagator.propagate(records, sources, use_multiprocessing=False)

mock_hits.df.head()

,time,string_id,module_id,pmt_id,record_id,type
0,6.681831,0,9,1,-3639979918762569235,cascade
1,12.337061,0,9,1,-3639979918762569235,cascade
2,7.632030,0,9,1,-3639979918762569235,cascade
3,8.672955,0,9,1,-3639979918762569235,cascade
4,14.230761,0,9,1,-3639979918762569235,cascade


And doing the same with the normal photons.

In [11]:
normal_hits = normal_photon_propagator.propagate(records, sources)

normal_hits.df.head()

/home/janik/projects/master/env/lib/python3.8/site-packages/haiku/_src/data_structures.py:206: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, structure = jax.tree_flatten(mapping)
/home/janik/projects/master/env/lib/python3.8/site-packages/haiku/_src/data_structures.py:206: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, structure = jax.tree_flatten(mapping)
/home/janik/projects/master/env/lib/python3.8/site-packages/haiku/_src/data_structures.py:206: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, structure = jax.tree_flatten(mapping)


,time,string_id,module_id,pmt_id,record_id,type
0,8.689281,0,7,4,-3639979918762569235,cascade
0,61.373062,0,8,2,-3639979918762569235,cascade
0,19.784624,0,8,4,-3639979918762569235,cascade
0,15.451541,0,8,9,-3639979918762569235,cascade
0,18.989155,0,8,10,-3639979918762569235,cascade


## Comparison of the photon propagators

Now that we have all the hits we want we can compare the following cases:

1. number of hits per module
2. hit arrival times


### Number of hits per module

Let's have a look at the mock propagation:

In [12]:
aggregated_mock_hits = mock_hits.df.set_index(['string_id', 'module_id'])
aggregated_mock_hits.groupby(level=[0,1]).count().head()

time  pmt_id  record_id  type
string_id module_id                               
0         9           211     211        211   211
          10         2738    2738       2738  2738
          11         9810    9810       9810  9810
          12         2247    2247       2247  2247
          13           66      66         66    66

In [13]:
aggregated_normal_hits = normal_hits.df.set_index(['string_id', 'module_id'])
aggregated_normal_hits.groupby(level=[0,1]).count().head()

time  pmt_id  record_id  type
string_id module_id                               
0         7             1       1          1     1
          8             7       7          7     7
          9            27      27         27    27
          10          223     223        223   223
          11         1620    1620       1620  1620

In [14]:
len(mock_hits)

15072

In [15]:
len(normal_hits)

3803